# Identication of putative contaminant taxa

In [1]:
setwd("/home/projects/14001280/PROJECTS/blood_microbiome/")
require(tidyverse)
require(ggplot2)
require(data.table)
require(egg)
require(compositions)

Loading required package: tidyverse

Warning message:
“Your system is mis-configured: ‘/etc/localtime’ is not a symlink”
Warning message:
“It is strongly recommended to set envionment variable TZ to ‘Asia/Singapore’ (or equivalent)”
── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.4     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: data.table


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘pack

### Load data

In [3]:
load_data <- function(file_path) {
    df <- fread(file_path) %>%
        separate(sample, into = c(NA, "npm_research_id"), sep = "_")
    return(df)
}

# genus_df <- load_data("data/temp_files/07_abundance_matrix/subset_100.G.tsv")
species_df <- load_data("data/temp_files/07_abundance_matrix/subset_100.S.tsv")

In [7]:
meta <- fread("data/SG10K_Health_metadata.n10714.16March2021.parsed.csv") %>%
    filter(npm_research_id %in% species_df$npm_research_id)
nrow(meta)
head(meta)

[1] 191

npm_research_id,multiplex_pool_id,supplier_id,gis_internal_sample_id,site_supplying_sample,year_of_birth,supplied_gender,self_reported_ethnicity,extraction_kit,date_of_dna_extraction,⋯,supplied_and_computed_ethinicity_match,sop_agreement_for_coverage__14/_28_for_15x/30x,sample_life_cycle,current,source_cohort,original_supplier_id,duplicate_info,duplicate,duplicate_pair,removal_requested_by_supplier
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,⋯,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>
WHB10062,MUX9707,00474,HELIOS_00474_1,HELIOS,1945,F,Chinese,abGENIX Whole Blood Genomic DNA Extraction Kit,6/6/2019,⋯,NA,Y,HELIOS VCF,Y,HELIOS,00474,,PASS,,NA
WHB10082,MUX9841,02947,HELIOS_02947_1,HELIOS,1970,F,Chinese,abGENIX Whole Blood Genomic DNA Extraction Kit,11/6/2019,⋯,NA,Y,HELIOS VCF,Y,HELIOS,02947,,PASS,,NA
WHB10100,MUX9844,02965,HELIOS_02965_1,HELIOS,1966,F,Chinese,abGENIX Whole Blood Genomic DNA Extraction Kit,11/6/2019,⋯,NA,Y,HELIOS VCF,Y,HELIOS,02965,,PASS,,NA
WHB10155,MUX9774,03025,HELIOS_03025_1,HELIOS,1979,F,Malay,abGENIX Whole Blood Genomic DNA Extraction Kit,11/6/2019,⋯,NA,Y,HELIOS VCF,Y,HELIOS,03025,,PASS,,NA
WHB10244,MUX9801,03118,HELIOS_03118_1,HELIOS,1980,M,Chinese,abGENIX Whole Blood Genomic DNA Extraction Kit,12/6/2019,⋯,NA,Y,HELIOS VCF,Y,HELIOS,03118,,PASS,,NA
WHB10342,MUX9870,03221,HELIOS_03221_1,HELIOS,1959,M,Malay,abGENIX Whole Blood Genomic DNA Extraction Kit,12/6/2019,⋯,NA,Y,HELIOS VCF,Y,HELIOS,03221,,PASS,,NA


### Retrieve samples that are present

In [12]:
retrieve_samples <- function(df) {
    df <- meta %>%
        inner_join(df, by = "npm_research_id")
    X <- df %>%
        select(colnames(.)[(ncol(meta) + 1):ncol(.)]) %>%
        as_tibble()
    Y <- df %>% 
        select(colnames(.)[1:ncol(meta)]) %>%
        as_tibble()
    
    return(list(X, Y))
}
# dat_genus <- retrieve_samples(genus_df)
# X_genus <- dat_genus[[1]]
# Y_genus <- dat_genus[[2]]

dat_species <- retrieve_samples(species_df)
X_species <- dat_species[[1]]
Y_species <- dat_species[[2]]

### Normalise

In [15]:
normalise <- function(df) {
    RA_df <- t(apply(df, 1, function(x) {x / sum(x) * 100}))
    return(as_tibble(RA_df))
}

# X_genus_norm <- normalise(X_genus)
X_species_norm <- normalise(X_species)

In [17]:
head(X_species)
Y_species

Pseudomonas mendocina,Pseudomonas aeruginosa,Pseudomonas pseudoalcaligenes,Pseudomonas furukawaii,Pseudomonas alcaligenes,Pseudomonas citronellolis,Pseudomonas resinovorans,Pseudomonas sp. LPH1,Pseudomonas alcaliphila,Pseudomonas sihuiensis,⋯,Desulfovibrio hydrothermalis,Bifidobacterium asteroides,Carboxydocella thermautotrophica,Capnocytophaga cynodegmi,Capnocytophaga sp. H2931,Bacteroides thetaiotaomicron,Gordonia phage Bantam,Natronolimnobius sp. AArc-Mg,Pseudoalteromonas phenolica,Anoxybacillus flavithermus
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,3,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,2,0,0,1,0,⋯,0,0,0,0,0,0,0,0,1,1
0,3,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,2,3,⋯,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,3,3,⋯,0,0,0,0,0,0,0,0,0,0
2,13,0,0,2,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


npm_research_id,multiplex_pool_id,supplier_id,gis_internal_sample_id,site_supplying_sample,year_of_birth,supplied_gender,self_reported_ethnicity,extraction_kit,date_of_dna_extraction,⋯,supplied_and_computed_ethinicity_match,sop_agreement_for_coverage__14/_28_for_15x/30x,sample_life_cycle,current,source_cohort,original_supplier_id,duplicate_info,duplicate,duplicate_pair,removal_requested_by_supplier
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,⋯,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>
WHB10062,MUX9707,00474,HELIOS_00474_1,HELIOS,1945,F,Chinese,abGENIX Whole Blood Genomic DNA Extraction Kit,6/6/2019,⋯,NA,Y,HELIOS VCF,Y,HELIOS,00474,,PASS,,NA
WHB10082,MUX9841,02947,HELIOS_02947_1,HELIOS,1970,F,Chinese,abGENIX Whole Blood Genomic DNA Extraction Kit,11/6/2019,⋯,NA,Y,HELIOS VCF,Y,HELIOS,02947,,PASS,,NA
WHB10100,MUX9844,02965,HELIOS_02965_1,HELIOS,1966,F,Chinese,abGENIX Whole Blood Genomic DNA Extraction Kit,11/6/2019,⋯,NA,Y,HELIOS VCF,Y,HELIOS,02965,,PASS,,NA
WHB10155,MUX9774,03025,HELIOS_03025_1,HELIOS,1979,F,Malay,abGENIX Whole Blood Genomic DNA Extraction Kit,11/6/2019,⋯,NA,Y,HELIOS VCF,Y,HELIOS,03025,,PASS,,NA
WHB10244,MUX9801,03118,HELIOS_03118_1,HELIOS,1980,M,Chinese,abGENIX Whole Blood Genomic DNA Extraction Kit,12/6/2019,⋯,NA,Y,HELIOS VCF,Y,HELIOS,03118,,PASS,,NA
WHB10342,MUX9870,03221,HELIOS_03221_1,HELIOS,1959,M,Malay,abGENIX Whole Blood Genomic DNA Extraction Kit,12/6/2019,⋯,NA,Y,HELIOS VCF,Y,HELIOS,03221,,PASS,,NA
WHB10417,MUX9829,03301,HELIOS_03301_1,HELIOS,1966,F,Indian,abGENIX Whole Blood Genomic DNA Extraction Kit,13/6/2019,⋯,NA,N,HELIOS VCF,Y,HELIOS,03301,,PASS,,NA
WHB10436,MUX9834,03321,HELIOS_03321_1,HELIOS,1953,M,Chinese,abGENIX Whole Blood Genomic DNA Extraction Kit,13/6/2019,⋯,NA,Y,HELIOS VCF,Y,HELIOS,03321,,PASS,,NA
WHB10470,MUX9808,03355,HELIOS_03355_1,HELIOS,1954,F,Chinese,abGENIX Whole Blood Genomic DNA Extraction Kit,13/6/2019,⋯,NA,Y,HELIOS VCF,Y,HELIOS,03355,,PASS,,NA


### Remove low abundance taxa

Please check that normalised values are 0-100% and not 0-1. Here we want to remove taxa that have less than a 1% maximum rel abundance across samples

In [22]:
remove_low_abundance <- function(df, threshold) {
    remove_df <- df %>%
        pivot_longer(everything(), names_to = "rank", values_to = "abundance") %>%
        group_by(rank) %>%
        summarise(max_abundance = max(abundance)) %>%
        filter(max_abundance > threshold)
    
    to_keep <- remove_df$rank
    print(paste0(length(to_keep), "/", ncol(df), " kept at threshold of ", threshold))
    return(colnames(df %>% select(all_of(to_keep))))
}

species_to_keep <- remove_low_abundance(X_species_norm, 0.5)
species_to_keep

[1] "282/2889 kept at threshold of 0.5"


[1] "Achromobacter denitrificans"                   
  [2] "Achromobacter insolitus"                       
  [3] "Achromobacter sp. AONIH1"                      
  [4] "Achromobacter spanius"                         
  [5] "Achromobacter xylosoxidans"                    
  [6] "Acidovorax sp. KKS102"                         
  [7] "Acinetobacter johnsonii"                       
  [8] "Actinoalloteichus sp. AHMU CJ021"              
  [9] "Actinomyces oris"                              
 [10] "Afipia sp. GAS231"                             
 [11] "Agrobacterium rhizogenes"                      
 [12] "Agrobacterium tumefaciens"                     
 [13] "Alcaligenes aquatilis"                         
 [14] "Alcaligenes faecalis"                          
 [15] "Alicycliphilus denitrificans"                  
 [16] "Altererythrobacter sp. B11"                    
 [17] "Aminobacter aminovorans"                       
 [18] "Aquabacterium olei"                            
 [19] "Bacillus anthracis"                            
 [20] "Bacillus cereus"                               
 [21] "Bacillus mycoides"                             
 [22] "Bacillus thuringiensis"                        
 [23] "Bdellovibrio bacteriovorus"                    
 [24] "Blastomonas fulva"                             
 [25] "Blastomonas sp. RAC04"                         
 [26] "Bordetella genomosp. 13"                       
 [27] "Bordetella genomosp. 8"                        
 [28] "Bosea sp. AS-1"                                
 [29] "Bosea sp. RAC05"                               
 [30] "Bosea sp. Tri-49"                              
 [31] "Bradyrhizobiaceae bacterium SG-6C"             
 [32] "Bradyrhizobium diazoefficiens"                 
 [33] "Bradyrhizobium erythrophlei"                   
 [34] "Bradyrhizobium icense"                         
 [35] "Bradyrhizobium japonicum"                      
 [36] "Bradyrhizobium lablabi"                        
 [37] "Bradyrhizobium oligotrophicum"                 
 [38] "Bradyrhizobium ottawaense"                     
 [39] "Bradyrhizobium sp."                            
 [40] "Bradyrhizobium sp. 2 39S1MB"                   
 [41] "Bradyrhizobium sp. 3 85S1MB"                   
 [42] "Bradyrhizobium sp. BTAi1"                      
 [43] "Bradyrhizobium sp. CCGE-LA001"                 
 [44] "Bradyrhizobium sp. ORS 278"                    
 [45] "Bradyrhizobium sp. ORS 285"                    
 [46] "Bradyrhizobium sp. ORS 3257"                   
 [47] "Bradyrhizobium sp. S23321"                     
 [48] "Bradyrhizobium sp. SK17"                       
 [49] "Bradyrhizobium sp. WSM471"                     
 [50] "Brevibacterium siliguriense"                   
 [51] "Brevundimonas sp. DS20"                        
 [52] "Brevundimonas sp. GW460-12-10-14-LB2"          
 [53] "Brevundimonas vesicularis"                     
 [54] "Burkholderia ambifaria"                        
 [55] "Burkholderia cenocepacia"                      
 [56] "Burkholderia cepacia"                          
 [57] "Burkholderia contaminans"                      
 [58] "Burkholderia lata"                             
 [59] "Burkholderia metallica"                        
 [60] "Burkholderia sp. NRF60-BP8"                    
 [61] "Burkholderia stabilis"                         
 [62] "Burkholderia vietnamiensis"                    
 [63] "Burkholderiales bacterium JOSHI_001"           
 [64] "Candidatus Planktophila lacus"                 
 [65] "Candidatus Planktophila sulfonica"             
 [66] "Caulobacter flavus"                            
 [67] "Caulobacter henricii"                          
 [68] "Caulobacter mirabilis"                         
 [69] "Caulobacter segnis"                            
 [70] "Caulobacter sp. K31"                           
 [71] "Caulobacter vibrioides"                        
 [72] "Celeribacter indicus"                          
 [73] "Chelativorans sp. BNC1"            

### Renormalise

### Test for batch effects

In [ ]:
test_batch_effects <- function(X, Y) {
    meta_cols <- colnames(meta)[grep("kit|flow_cell|instrument_id|source", colnames(meta))]
    taxa <- colnames(X)

    result_df <- tibble()

    for (col in meta_cols) {
        for (taxon in taxa) {
#             X <- X_genus
#             Y <- Y_genus
#             taxon <- taxa[6]
#             col <- meta_cols[5]
            x <- X[, taxon]
            y <- Y[, col]
            test_df <- cbind(x, y)

            # Remove samples with unknown or NA metadata
            test_filt <- test_df %>%
                filter(!grepl("unknown", get(col), ignore.case = T) & !is.na(get(col)))
            
            # Remove factors that do not have any reads
            reads <- test_filt %>%
                group_by(get(col)) %>%
                summarise(max_reads = max(get(taxon))) %>%
                filter(max_reads != 0)
            
            to_keep <- reads$`get(col)`
            
            if (length(to_keep) >= 2) {
                test_filt <- test_filt %>%
                    filter(get(col) %in% to_keep)

                # One-way ANOVA
                a <- aov(as_vector(test_filt[, taxon]) ~ as_vector(test_filt[, col]))
                summ <- unlist(summary(a))
                deg_f <- summ["Df1"]
                F_stat <- summ["F value1"]
                p_val <- summ["Pr(>F)1"]

                # Save results
                morsel <- tibble(taxon = taxon, meta_name = col, df = deg_f, F_stat = F_stat, p_val = p_val)
                result_df <- result_df %>% bind_rows(morsel)
            } else {
#                 print(paste("Skipped", taxon, col))
#                 reads
#                 to_keep
                
            }
        }
    }
    
    return(result_df)
}

genus_result <- test_batch_effects(X_genus, Y_genus)
species_result <- test_batch_effects(X_species, Y_species)

In [ ]:
genus_result
species_result

### Retain genera whose smallest p-value is greater than t

we might want to correct the threshold for multiple testing

In [ ]:
t <- 0.05

In [ ]:
length(unique(genus_result$taxon))
length(unique(species_result$taxon))

In [ ]:
filter_results <- function(result_df) {
    filt_df <- result_df %>% 
        group_by(taxon) %>%
        summarise(smallest_p_val = min(p_val)) %>%
        filter(smallest_p_val > t) %>%
        mutate(log_p_val = -log(smallest_p_val, base = 10), 
               taxon = fct_reorder(taxon, log_p_val, .desc = F))
    return(filt_df)
}

plot_genus <- filter_results(genus_result)
plot_species <- filter_results(species_result)

In [ ]:
length(unique(plot_genus$taxon))
length(unique(plot_species$taxon))

### Visualise results

In [ ]:
plot_results <- function(filt_df, taxon_name, subset) {    
    if (is.na(subset)) {
        to_plot <- filt_df
    } else {
        to_plot <- filt_df %>%
        arrange(log_p_val) %>%
        slice(1:subset)
    }
    
    plt <- to_plot %>%
        ggplot(aes(x = taxon, y = log_p_val, fill = log_p_val)) +
        geom_bar(stat = "identity") + 
        labs(x = taxon_name, y = "-lg(minimum p-value)") +
        theme(axis.text.x = element_text(angle = 45, hjust = 1),
              legend.position = "none",
              plot.margin = margin(t = 0, r = 0, b = 0, l = 2, unit = "cm"))
    
    return(plt)
}

plot_results(plot_genus, "Genus", NA)
plot_results(plot_species, "Species", 30)

## Sanity check

#### Genera flagged as non-contaminants

In [ ]:
join_results <- function(df, plot_df) {
    max_df <- df %>%
        pivot_longer(everything(), names_to = "rank", values_to = "abundance") %>%
        group_by(rank) %>%
        summarise(max_abundance = max(abundance))

    plot_df %>%
        rename(rank = taxon) %>%
        left_join(max_df, "rank") %>%
        arrange(log_p_val)
}

join_results(X_genus, plot_genus)
join_results(X_species, plot_species)

#### Genera flagged as contaminants

In [ ]:
genus_result %>%
    filter(meta_name == "source_cohort") %>%
    arrange(p_val)

#### Plot per-batch RA of contaminants vs. non-contaminants

In [ ]:
plot_sanity_plot <- function(tax) {
    print(tax)
    temp_plt <- cbind(X_genus, Y_genus) %>%
        mutate(log_abundance = log(get(tax) + 0.0001)) %>%
        ggplot(aes(x = source_cohort, y = get(tax), fill = source_cohort)) + 
        geom_point() +
        geom_boxplot(alpha = 0.5) +
        labs(x = "Source cohort", y = paste(tax, "% abundance")) +
        theme(legend.position = "none",
          axis.text.x = element_text(angle = 45, hjust = 1),
          text = element_text(size = 15))
    return(temp_plt)
}

In [ ]:
plot_sanity_plot("Pseudomonas")
plot_sanity_plot("Klebsiella")
plot_sanity_plot("Brachybacterium")


In [ ]:
plot_log_sanity_plot <- function(tax) {
    print(tax)
    temp_plt <- cbind(X_genus, Y_genus) %>%
        mutate(log_abundance = log(get(tax) + 0.0001, base = 10)) %>%
        ggplot(aes(x = source_cohort, y = log_abundance, fill = source_cohort)) + 
        geom_point() +
        geom_boxplot(alpha = 0.5) +
        labs(x = "Source cohort", y = paste(tax, "lg(% abundance + 0.0001)")) +
        theme(legend.position = "none",
          axis.text.x = element_text(angle = 45, hjust = 1),
          text = element_text(size = 15))
    return(temp_plt)
}

plot_log_sanity_plot("Klebsiella")